In [ ]:
!pip install instructor

In [ ]:
!pip install python-dotenv

In [15]:
from domain.entities.pet import Pet
import instructor 
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(".env")

True

In [16]:
from domain.entities.pet import Pet
from domain.attributes.pet_characteristic import PetCharacteristic
from domain.constants.attributes import (
    ActivityLevel, Size, Temperament, FurType, IntelligenceLevel,
    MaintenanceLevel, Sex
)
pet = Pet(
    name="Buddy",
    species="Dog",
    breed="Labrador Retriever",
    sex=Sex.MALE,
    age=3,
    petChar=PetCharacteristic(
        activity_level=ActivityLevel.HIGH,
        size=Size.LARGE,
        temperament=Temperament.CALM,
        fur_type=FurType.SHORT,
        intelligence=IntelligenceLevel.HIGH,
        maintenance=MaintenanceLevel.MEDIUM,
        breed="Labrador Retriever",
        species="Dog",
        hypoallergenic="N"
    )
)

In [28]:
system_prompt = """
You are an intelligent assistant helping to map user pet descriptions into a structured survey model.

Analyze the user's text carefully.

Follow these rules:
- If the text mentions a pet playing, running, or being active, assume moderate running miles.
- If the pet is small, loves cuddling, and smells good, assume grooming needs are medium and couch fur happiness is high (4 or 5).
- If pet size is mentioned as small, set 'happy_with_small_dogs' = True. If no mention of large dogs, default 'happy_with_large_dogs' = False.
- If the pet loves cuddling and being near people, set 'personality_traits' to friendly and loyal.
- If the description shows active play (balls, toys, running), assume 'active_dogs_enjoyment' = VERY_MUCH.
- If the text shows strong affection (cuddles, treats), set 'cute_dogs' = VERY_MUCH.
- If not specified, default vacuuming to ONCE per week, yard to True, and food spending to $200.
- If the pet doesn't shed hair then assume the pet is hypoallergenic

Important:
- Do not fabricate facts not hinted at in the text.
- If unsure, leave optional fields (like 'intact_requirement') as None.

Output the structured PetAdoptionSurvey fields accordingly.
"""


In [29]:

openai_client = OpenAI(api_key=os.getenv("OPEN_AI_KEY"))

client = instructor.from_openai(openai_client)

# Extract structured data from natural language
res = client.chat.completions.create(
    model="gpt-4o-mini",
    response_model=PetCharacteristic,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{sample_survey_1}"}
    ],
)


In [30]:

with open("output.json", "w") as f:
    f.write(res.model_dump_json(indent=4))


In [24]:
from backend.app.services.questionaire import PetAdoptionSurvey, GroomingSpending, RunningMiles, VacuumTimes, TravelDistance, EnvisionedAge, JourneyPayment, ActiveDogsEnjoyment, CompatibilityValue, CuteDogsLove, OtherPets, GenderPreference

sample_survey = PetAdoptionSurvey(
    grooming_spending=GroomingSpending.MEDIUM,
    running_miles=RunningMiles.RANGE_2_5,
    couch_fur_happiness=4,
    vacuum_times=VacuumTimes.ONE,
    happy_with_large_dogs=True,
    happy_with_small_dogs=True,
    hoa_pet_contract="https://example.com/hoa_pet_contract.png",
    other_pets=[OtherPets.CAT, OtherPets.EXOTIC],
    kids_around_friend=True,
    travel_distance=TravelDistance.FIVE_TO_TEN,
    home_address="1234 Bark Street, Dogtown, USA",
    paid_transport=True,
    envisioned_age=EnvisionedAge.TWELVE_TO_THIRTY_SIX,
    plan_to_travel=True,
    journey_payment=JourneyPayment.MEDIUM,
    food_spending=200,
    has_yard=True,
    personality_traits="Friendly, playful, loyal",
    active_dogs_enjoyment=ActiveDogsEnjoyment.VERY_MUCH,
    value_compatibility=CompatibilityValue.VERY_MUCH,
    cute_dogs=CuteDogsLove.VERY_MUCH,
    intact_requirement=None,  # Doesn't matter
    only_rescue=False,
    gender_preference=GenderPreference.FEMALE,
)

sample_survey_1 = PetAdoptionSurvey(
    grooming_spending=GroomingSpending.LOW,
    running_miles=RunningMiles.RANGE_0_1,
    couch_fur_happiness=5,
    vacuum_times=VacuumTimes.ONE,
    happy_with_large_dogs=True,
    happy_with_small_dogs=True,
    hoa_pet_contract="https://example.com/hoa1.png",
    other_pets=[OtherPets.CAT],
    kids_around_friend=True,
    travel_distance=TravelDistance.TEN_PLUS,
    home_address="123 Elm Street, Springfield, IL",
    paid_transport=False,
    envisioned_age=EnvisionedAge.FIVE_TO_TWELVE,
    plan_to_travel=True,
    journey_payment=JourneyPayment.HIGH,
    food_spending=200,
    has_yard=True,
    personality_traits="Playful, friendly, energetic",
    active_dogs_enjoyment=ActiveDogsEnjoyment.VERY_MUCH,
    value_compatibility=CompatibilityValue.VERY_MUCH,
    cute_dogs=CuteDogsLove.VERY_MUCH,
    only_rescue=True,
    gender_preference=GenderPreference.MALE
)

